In [1]:
import pandas as pd
import numpy as np

In [2]:
from tqdm.notebook import tqdm

In [3]:
train = pd.read_json('../data/train.json')

In [4]:
train.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000


In [5]:
# songs = pd.read_json('../data/song_meta.json')

In [6]:
# songs.head()

In [7]:
# genre = pd.read_json('../data/genre_gn_all.json', typ='series')

In [8]:
# genre = pd.DataFrame(genre).reset_index().rename(columns={'index':'gn', 0:'desc'})

In [7]:
val = pd.read_json('../data/val.json')

In [8]:
val.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000
3,[],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000
4,[],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000


In [10]:
train.shape, val.shape

((115071, 6), (23015, 6))

## validation data cases - no songs

In [154]:
val['num_songs'] = val['songs'].map(lambda x: len(x))

In [155]:
# 곡이 없는 경우 4370건
# 0.1898, 약 19%
val[val['num_songs']==0]

,tags,id,plylst_title,songs,like_cnt,updt_date,num_songs
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000,0
8,[스트레스],80810,리듬타면서 빡시게 운동하자!!!(스트레스 날리자):},[],127,2017-02-09 17:33:45.000,0
9,[],142007,기분 좋은 재즈와 함께 만드는 달달한 하루,[],0,2015-06-22 09:11:02.000,0
17,[생각나],2380,다시 생각나는 그 사람,[],6,2016-03-19 23:53:20.000,0
20,"[피아노, 이루마, 메로디]",149069,불면증엔 아름다운 뉴에이지 곡들,[],11,2019-12-15 19:15:21.000,0
...,...,...,...,...,...,...,...
22991,[],32537,컨트리 황제 조니 캐시가 선 레코드 시절 발표한 초기 대표작,[],28,2019-06-17 14:22:48.000,0
22992,[사랑],32812,♥옷차림이 가벼워질때 부담없이 듣는음악♥,[],8,2015-05-08 00:09:01.000,0
22996,[],86721,해 저무는 밤,[],5,2016-04-27 15:32:55.000,0
23002,"[텐션, 명곡, 학창시절, 텐션업]",56570,숨어듣는명곡! 텐션올릴때듣는 그시절그노래!! (그냥명곡있을수있음주의),[],538,2020-04-18 22:02:07.000,0


# CF model
* Playlist를 사람, songs를 items로

## [추후, 지금은 제외] preprocessing - duplicated rows(subset on tags and songs)
* 중복된 플레이리스트 중, like_cnt가 높은 row만 남기기

In [12]:
# temp = train.copy()
# temp.sort_values(by='like_cnt', ascending=False, inplace=True)
# temp['tags'] = temp['tags'].astype('str')
# temp['songs'] = temp['songs'].astype('str')

# temp.drop_duplicates(subset=['tags','songs'], keep='first', inplace=True)
# nondup_index = temp.index
# train = train.iloc[nondup_index, :]

# del nondup_index, temp

# train.reset_index(drop=True, inplace=True)

## preprocessing - CF dataframe TRAIN

In [11]:
from scipy.sparse import csr_matrix

In [101]:
data, rows, cols = [], [], []

for a, b in tqdm(train[['id','songs']].values):
    for bs in b:
        rows.append(a)
        cols.append(bs)
        data.append(1)

In [102]:
train_csr = csr_matrix((data, (rows, cols)), shape=(train.id.max()+1,707989))

In [14]:
print(train_csr.count_nonzero())

5285871


In [21]:
# train_dense = train_csr.todense()

In [22]:
# print(train_dense.shape)

In [17]:
print(train.id.min(), train.id.max())

1 153428


## preprocessing - CF datafram VAL

In [18]:
val.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000
3,[],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000
4,[],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000


In [129]:
print(val.id.min(), val.id.max())

3 153424


In [103]:
data, rows, cols = [], [], []

for a, b in tqdm(val[['id','songs']].values):
    for bs in b:
        rows.append(a)
        cols.append(bs)
        data.append(1)

In [104]:
val_csr = csr_matrix((data, (rows, cols)), shape=(val.id.max()+1,707989))

In [24]:
# val_dense = val_csr.todense()

### intersection
* val_csr[3] 기준

In [132]:
intersection = val_csr[3].dot(train_csr.T)

In [133]:
print(intersection.shape)

(1, 153429)


In [134]:
intersection[0,0]

0

In [135]:
for i in range(0, 153429):
    if intersection[0, i]!=0:
        print(i, intersection[0, i])

20 1
25 1
47 1
51 1
96 1
110 2
117 1
118 1
140 1
141 5
157 1
158 1
167 2
185 1
197 1
199 1
215 1
218 1
227 2
228 1
266 3
289 1
318 2
325 4
334 1
343 1
366 1
389 1
395 1
398 1
409 1
417 3
419 1
420 1
422 1
429 1
440 2
441 1
467 1
473 2
491 1
509 1
513 6
525 4
527 1
537 1
542 2
545 2
550 1
579 3
582 1
587 3
589 1
590 1
619 2
695 1
713 1
719 1
726 1
740 1
759 1
769 1
783 1
801 5
852 2
925 1
966 1
969 1
973 3
976 1
1002 1
1038 1
1045 4
1076 1
1082 1
1092 1
1099 1
1104 1
1110 1
1124 1
1148 1
1156 1
1158 2
1172 2
1209 2
1212 1
1216 1
1219 1
1231 1
1233 7
1235 1
1246 1
1257 1
1294 2
1312 1
1322 2
1331 3
1338 1
1339 1
1349 4
1368 2
1378 1
1386 1
1402 1
1408 1
1412 1
1418 1
1428 3
1433 2
1442 2
1448 2
1449 1
1467 1
1479 1
1485 1
1491 2
1502 1
1521 1
1523 1
1529 1
1541 1
1550 1
1615 1
1623 1
1644 1
1658 1
1661 4
1664 1
1673 2
1674 2
1677 1
1689 5
1692 2
1694 1
1703 1
1710 1
1723 1
1753 1
1754 1
1772 3
1781 1
1788 2
1791 1
1803 3
1806 1
1832 1
1837 1
1838 1
1841 1
1850 1
1855 1
1874 1
1879 4
1885

14635 1
14655 2
14691 1
14697 2
14713 2
14717 3
14719 2
14751 1
14752 2
14762 2
14773 1
14790 1
14791 2
14808 3
14812 1
14817 2
14837 1
14841 1
14851 3
14859 1
14881 1
14883 1
14901 1
14906 2
14916 1
14927 3
14933 2
14938 1
14949 1
14957 1
14960 2
14977 1
14986 1
15014 1
15021 1
15024 1
15030 2
15031 1
15039 1
15047 2
15048 1
15060 1
15061 1
15066 1
15070 1
15076 1
15089 1
15100 2
15101 1
15122 1
15126 1
15157 1
15159 1
15217 1
15228 1
15239 1
15258 1
15277 1
15281 1
15300 2
15306 1
15313 1
15355 1
15373 1
15375 2
15393 1
15395 1
15396 1
15416 1
15423 1
15429 1
15436 1
15446 3
15454 2
15455 2
15484 2
15493 2
15496 1
15502 1
15509 2
15525 3
15538 2
15546 1
15553 2
15563 1
15564 1
15567 1
15570 1
15623 3
15634 1
15644 1
15661 1
15674 2
15680 1
15686 1
15687 1
15692 5
15707 3
15710 1
15714 1
15742 1
15746 3
15751 1
15756 4
15781 2
15783 1
15784 1
15786 1
15787 1
15791 3
15800 1
15826 1
15829 1
15830 2
15832 2
15838 1
15859 1
15861 1
15882 1
15887 2
15891 1
15895 2
15904 1
15911 1
15914 1


25842 1
25857 1
25862 1
25870 1
25880 1
25882 1
25898 1
25913 1
25931 1
25939 1
25952 2
25957 1
25971 1
25973 1
25974 1
25992 1
26007 1
26016 1
26029 1
26034 1
26036 4
26060 2
26064 1
26070 1
26074 1
26076 2
26096 1
26099 1
26110 1
26123 1
26142 5
26162 1
26163 1
26167 2
26182 1
26223 1
26229 1
26230 2
26247 1
26265 1
26282 1
26285 1
26288 1
26298 1
26326 1
26337 2
26342 1
26347 1
26349 1
26355 1
26391 1
26393 1
26404 1
26442 1
26443 2
26445 1
26447 1
26456 1
26467 1
26475 1
26476 2
26485 2
26499 2
26521 1
26526 1
26536 1
26538 1
26546 1
26559 1
26564 2
26573 4
26584 2
26602 3
26625 2
26628 5
26638 1
26653 1
26655 1
26658 1
26679 1
26701 4
26702 1
26727 1
26738 1
26743 1
26755 1
26758 1
26759 1
26770 1
26773 1
26783 1
26784 1
26796 1
26811 2
26813 1
26816 1
26836 1
26841 3
26842 1
26845 2
26892 1
26904 1
26909 1
26912 1
26920 5
26922 1
26923 1
26938 1
26941 1
26944 1
26956 1
26957 1
26971 2
26983 2
26987 2
26990 1
26999 1
27010 4
27011 2
27054 1
27074 1
27075 3
27094 1
27120 1
27135 2


36895 2
36899 1
36900 1
36910 1
36916 1
36926 4
36932 1
36940 1
36943 1
36962 1
36970 1
36982 2
36997 1
37003 1
37012 2
37013 2
37014 2
37026 2
37029 1
37044 1
37053 1
37056 1
37059 1
37064 1
37086 1
37094 1
37097 1
37101 1
37108 1
37126 1
37134 2
37139 1
37155 1
37157 2
37169 1
37170 1
37171 2
37178 3
37181 1
37183 1
37191 1
37201 1
37214 1
37222 3
37225 1
37226 1
37234 1
37260 2
37263 3
37266 1
37276 3
37285 3
37296 1
37319 1
37322 1
37331 2
37337 1
37342 1
37368 1
37386 1
37387 1
37396 2
37401 1
37404 1
37417 1
37423 1
37464 2
37488 3
37491 1
37497 1
37498 2
37522 2
37525 4
37529 1
37542 2
37572 2
37581 1
37592 1
37600 1
37603 1
37625 2
37647 1
37681 2
37720 1
37723 1
37729 1
37734 1
37739 2
37742 1
37746 2
37748 1
37752 2
37754 1
37783 3
37798 1
37805 1
37829 1
37837 1
37838 2
37850 1
37863 1
37866 1
37871 1
37877 1
37878 1
37889 1
37910 2
37931 1
37935 2
37937 1
37970 2
37988 2
38001 1
38015 1
38030 1
38037 1
38048 1
38049 3
38062 2
38067 1
38071 1
38072 3
38076 1
38100 1
38103 3


50613 3
50619 1
50626 1
50655 1
50659 12
50660 1
50663 1
50671 1
50675 1
50677 3
50681 1
50687 2
50694 1
50716 3
50728 2
50731 2
50733 1
50741 1
50750 2
50751 1
50765 1
50777 1
50785 1
50790 1
50792 1
50819 1
50827 1
50838 3
50866 1
50895 2
50896 1
50913 2
50925 1
50948 1
50952 1
50953 1
50955 1
50963 1
50988 2
51006 1
51022 1
51032 1
51039 1
51061 1
51083 1
51087 1
51090 1
51108 1
51144 1
51145 1
51152 1
51156 1
51162 1
51163 1
51168 1
51195 3
51199 1
51223 1
51226 1
51231 1
51232 2
51237 5
51269 1
51277 1
51283 2
51291 1
51294 1
51307 1
51315 1
51318 1
51329 3
51338 3
51378 1
51388 3
51393 1
51405 1
51417 2
51439 1
51444 1
51468 2
51486 1
51489 2
51491 2
51496 2
51498 1
51525 1
51531 1
51546 1
51552 1
51563 1
51585 4
51593 1
51599 1
51611 3
51632 2
51634 1
51638 1
51650 1
51651 1
51659 1
51674 1
51675 3
51682 1
51684 1
51732 1
51748 1
51752 1
51775 1
51779 1
51810 1
51827 4
51830 2
51838 4
51843 1
51845 1
51851 1
51855 3
51859 1
51861 1
51877 1
51878 1
51887 1
51889 2
51895 1
51908 1

...

97950 3
97954 1
97970 1
97971 1
97977 3
97981 1
97994 1
98002 1
98021 1
98025 1
98050 1
98052 1
98054 1
98059 1
98070 1
98071 1
98074 2
98077 1
98128 1
98141 1
98143 4
98146 1
98155 3
98177 1
98212 1
98217 1
98235 1
98237 4
98239 1
98259 1
98261 1
98267 3
98286 1
98296 1
98297 1
98298 1
98306 1
98332 1
98369 1
98379 3
98380 1
98390 1
98401 1
98413 1
98420 2
98432 3
98444 1
98466 1
98469 1
98477 1
98493 1
98513 2
98518 4
98519 1
98526 1
98542 1
98563 2
98564 3
98585 2
98586 1
98597 1
98605 5
98614 1
98644 1
98656 2
98658 2
98686 1
98691 1
98708 2
98713 2
98736 2
98739 1
98744 1
98745 1
98755 2
98775 3
98786 2
98821 1
98845 3
98860 1
98864 1
98874 1
98880 2
98886 1
98895 1
98900 1
98905 1
98919 1
98921 1
98937 3
98953 1
98959 1
98964 2
98971 1
98997 2
99025 1
99057 1
99071 3
99078 3
99081 1
99106 1
99122 1
99135 2
99138 1
99143 2
99156 1
99168 10
99179 2
99188 1
99189 1
99200 1
99206 1
99215 2
99225 1
99244 1
99253 2
99263 1
99266 1
99271 1
99284 1
99338 1
99356 1
99360 1
99365 2
99380 1

109034 1
109042 1
109043 1
109052 1
109053 1
109059 1
109061 1
109084 3
109099 3
109105 1
109106 2
109108 1
109112 3
109120 1
109122 3
109151 1
109156 2
109160 4
109161 1
109167 1
109169 1
109186 2
109191 2
109200 1
109203 1
109210 1
109213 1
109218 1
109223 4
109224 1
109228 2
109233 1
109234 1
109258 1
109269 2
109291 1
109292 1
109295 1
109301 1
109309 1
109322 1
109370 1
109387 1
109402 1
109423 1
109427 2
109428 1
109447 2
109448 1
109459 1
109482 2
109490 2
109498 1
109507 5
109522 1
109533 4
109539 5
109546 3
109565 1
109579 4
109584 1
109599 1
109602 1
109603 1
109606 1
109638 1
109644 1
109656 1
109668 1
109670 2
109676 1
109692 1
109698 1
109711 1
109732 1
109756 1
109779 2
109810 1
109821 1
109835 1
109840 2
109845 1
109846 1
109857 1
109887 1
109892 1
109894 1
109906 1
109910 2
109915 1
109920 2
109923 2
109933 1
109949 1
109954 1
109969 1
109984 1
109991 1
109995 1
110004 1
110006 5
110009 1
110019 2
110040 1
110050 1
110060 2
110077 1
110101 2
110107 1
110113 2
110117 1
1

120512 1
120526 4
120536 2
120549 1
120555 1
120557 1
120561 1
120588 1
120595 1
120597 4
120608 5
120626 1
120633 1
120635 1
120637 1
120642 1
120664 1
120670 3
120678 1
120685 1
120686 1
120689 1
120692 1
120717 5
120719 1
120732 1
120735 1
120768 2
120769 2
120773 1
120774 1
120775 1
120780 1
120801 3
120806 1
120809 2
120812 1
120813 1
120814 1
120850 2
120863 1
120884 2
120890 1
120896 1
120901 1
120919 2
120939 2
120944 1
120952 1
120967 1
120988 1
120995 1
121002 1
121004 1
121011 3
121029 1
121034 2
121038 1
121047 1
121058 1
121069 1
121070 3
121106 1
121120 1
121121 2
121127 3
121129 1
121131 2
121142 4
121155 1
121165 1
121185 1
121188 3
121212 2
121229 1
121238 1
121240 1
121246 3
121254 1
121257 1
121262 1
121267 3
121280 1
121285 3
121286 2
121291 1
121292 1
121295 2
121332 1
121357 1
121364 2
121365 1
121408 2
121409 1
121419 4
121432 1
121438 5
121448 1
121462 2
121467 5
121476 2
121492 1
121500 4
121506 1
121514 1
121524 4
121529 1
121533 1
121538 2
121567 1
121579 1
1

131693 2
131729 1
131734 1
131761 2
131765 1
131772 1
131775 1
131784 1
131798 2
131805 2
131815 1
131833 3
131845 3
131875 1
131892 1
131905 1
131940 1
131949 1
131961 1
131973 5
131996 1
132009 1
132015 1
132021 1
132033 2
132037 1
132050 2
132054 1
132062 2
132072 1
132081 2
132088 1
132090 3
132094 1
132107 1
132109 1
132123 1
132125 1
132126 1
132163 3
132192 1
132208 1
132224 1
132254 1
132256 1
132266 1
132294 2
132296 2
132303 2
132319 1
132327 1
132346 1
132347 1
132350 1
132363 1
132387 1
132416 2
132422 1
132429 1
132437 1
132453 1
132463 2
132467 1
132473 1
132477 1
132523 1
132535 1
132560 1
132570 1
132581 1
132588 2
132625 1
132633 1
132637 1
132641 1
132646 1
132654 1
132655 1
132660 1
132663 1
132667 2
132668 1
132684 5
132705 1
132712 4
132734 2
132740 1
132755 1
132779 1
132803 2
132836 1
132855 1
132892 1
132904 2
132969 1
132971 1
132972 2
133001 1
133012 1
133014 1
133015 1
133018 4
133028 1
133030 1
133031 1
133036 1
133046 1
133067 1
133073 2
133082 4
133113 1
1

143090 1
143108 1
143138 2
143146 2
143148 1
143151 2
143155 1
143166 1
143168 3
143176 1
143179 2
143185 1
143192 1
143199 1
143207 3
143209 1
143210 2
143215 1
143217 1
143273 1
143289 1
143292 1
143298 1
143315 1
143321 1
143333 2
143340 1
143351 1
143366 1
143372 1
143388 3
143390 1
143413 1
143417 1
143426 1
143427 1
143429 1
143432 1
143443 2
143452 2
143464 1
143466 1
143482 1
143495 1
143502 1
143529 4
143545 5
143573 1
143576 1
143582 1
143600 1
143630 1
143640 1
143642 1
143680 4
143682 1
143688 1
143696 1
143708 2
143712 1
143713 1
143715 2
143756 2
143768 2
143770 1
143801 4
143802 1
143812 2
143821 1
143826 1
143827 1
143847 1
143853 1
143866 1
143869 1
143886 4
143887 1
143913 1
143929 1
143932 1
143944 4
143951 2
143955 1
143956 1
143958 1
143960 4
143963 3
143967 2
143977 1
143979 3
144007 2
144018 1
144019 1
144030 2
144034 1
144038 1
144044 1
144045 3
144057 3
144060 2
144063 1
144066 1
144070 3
144073 6
144093 1
144099 1
144110 1
144116 1
144127 1
144140 1
144147 3
1

### union

In [128]:
# train.id.min() == 1
train_csr[1].sum()

5

In [130]:
# val.id.min() == 3
val_csr[3].sum()

66

In [123]:
# boolean type 변경
train_csr_ = train_csr.astype('bool')
val_csr_ = val_csr.astype('bool')

In [131]:
(train_csr_[1] + val_csr_[3]).sum()

71

### jaccard score

In [137]:
intersection[0,20]/((train_csr_[20]+val_csr_[3]).sum())

0.011904761904761904

In [138]:
intersection[0,141]/((train_csr_[141]+val_csr_[3]).sum())

0.024630541871921183

In [139]:
intersection[0,2638]/((train_csr_[2638]+val_csr_[3]).sum())

0.02459016393442623

### recommend

In [180]:
import itertools

In [161]:
# val_csr[3] 기준 예시
scores = {}
intersection = val_csr[3].dot(train_csr.T)

for i in tqdm(range(0, 153429)):
    if intersection[0, i]==0:
        pass
    else:
        jd = intersection[0, i]/((train_csr_[i]+val_csr_[3]).sum())
        scores[i] = jd

In [166]:
max(scores.values())

0.08411214953271028

In [167]:
min(scores.values())

0.0037735849056603774

In [169]:
scores = sorted(scores.items(), reverse=True, key=lambda x: x[1])

In [184]:
given = list(itertools.chain(*list(val[val.id==3].songs)))

In [216]:
result = []
for i in scores:
    basket = list(itertools.chain(*list(train[train.id==i[0]].songs)))
    for item in basket:
        if len(result)==100:
            break
        if item not in given:
            result.append(item)
        print(len(result))

0
1
2
3
4
4
5
6
7
8
9
10
11
12
13
14
15
16
16
17
18
19
20
21
22
23
24
25
26
26
27
28
28
29
29
29
30
31
32
32
33
34
35
36
37
38
39
40
41
41
41
41
42
43
44
44
45
46
47
48
49
49
50
51
52
53
53
54
55
55
56
56
56
57
58
59
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
90
91
91
91
92
93
94
94
95
96
97
98
99
100


In [217]:
len(result)

100

In [218]:
result

[576231,
 698865,
 351342,
 73547,
 376358,
 675500,
 183579,
 678936,
 280830,
 473756,
 462446,
 292825,
 688785,
 64052,
 216696,
 375450,
 228031,
 99948,
 133190,
 244808,
 155356,
 367459,
 388758,
 196664,
 160214,
 321564,
 116573,
 241272,
 177284,
 698250,
 528130,
 505036,
 518420,
 627363,
 37748,
 11914,
 548735,
 462965,
 411756,
 52904,
 641599,
 376358,
 675500,
 183579,
 367459,
 388758,
 196664,
 160214,
 321564,
 292825,
 688785,
 64052,
 375450,
 228031,
 116573,
 177284,
 698250,
 528130,
 505036,
 518420,
 627363,
 37748,
 318576,
 11914,
 99948,
 133190,
 244808,
 155356,
 241272,
 576231,
 698865,
 351342,
 73547,
 90733,
 678936,
 65773,
 280830,
 548735,
 462965,
 411756,
 52904,
 641599,
 473756,
 462446,
 698865,
 351342,
 73547,
 678936,
 280830,
 116573,
 177284,
 698250,
 528130,
 505036,
 518420,
 627363,
 37748,
 11914,
 548735,
 462965]

## recommend function

In [221]:
def recommend_100(val_id):
    scores = {} 
    intersection = val_csr[val_id].dot(train_csr.T)

    for i in tqdm(range(0, 153429)):
        if intersection[0, i]==0:
            pass
        else:
            jd = intersection[0, i]/((train_csr_[i]+val_csr_[val_id]).sum())
            scores[i] = jd
            
    scores = sorted(scores.items(), reverse=True, key=lambda x: x[1])
    
    given = list(itertools.chain(*list(val[val.id==3].songs)))
    result = []
    for i in scores:
        basket = list(itertools.chain(*list(train[train.id==i[0]].songs)))
        for item in basket:
            if len(result)==100:
                break
            if item not in given:
                result.append(item)
    return result

In [222]:
example = recommend_100(118598)

In [223]:
example

[415351,
 566502,
 412246,
 660172,
 187531,
 61612,
 151421,
 516376,
 219415,
 551157,
 654428,
 467269,
 164168,
 416023,
 680970,
 557956,
 411438,
 646988,
 157900,
 320447,
 19606,
 425357,
 219971,
 21512,
 690767,
 438915,
 149081,
 271335,
 653565,
 125214,
 328908,
 333970,
 12795,
 107413,
 162569,
 161136,
 66475,
 623047,
 367826,
 209622,
 207912,
 586389,
 292859,
 328517,
 310892,
 15774,
 381851,
 225652,
 608704,
 261743,
 268669,
 562903,
 295701,
 461170,
 78983,
 76044,
 127698,
 352137,
 243541,
 269427,
 455806,
 703323,
 638488,
 422438,
 592526,
 413459,
 545816,
 693580,
 359986,
 611201,
 520034,
 412305,
 110696,
 350062,
 472374,
 104455,
 520437,
 373313,
 151080,
 656598,
 288665,
 694423,
 308797,
 637900,
 262430,
 551161,
 434094,
 149051,
 607850,
 101286,
 620311,
 584108,
 696876,
 165237,
 525935,
 491808,
 5418,
 340089,
 651465,
 303904]